In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
import numpy as np
from export.saver import read_pickle
from helpers.layout import Layout
from helpers.graph_helpers import *
from helpers.helpers import sort_and_group_objects
from typing import Dict
from domains.domain import Domain
from helpers.directions import get_axis, get_opposite_axis, Direction
from placement2.arrange import *
from placement2.place import *
from copy import copy, deepcopy
from icecream import ic
from visuals.plotter import plot_general
from itertools import product
from fixes.reporter import Reporter
from new_solutions.selection import Cook
from domains.range import Range



In [3]:
layout: Layout = read_pickle("1001_amber_c_ag")
G = layout.graph
domains = layout.domains

In [4]:
arr = create_arrangement(layout)
darr = create_domains_arr(arr, layout.domains)
darr1 = place_nodes(darr)

arrange> NORTH EAST node added when processing 2: ['corridor']
arrange> NORTH EAST node added when processing 3: ['corridor']


In [5]:
doms = create_new_domains(darr1)
doms

{'m_bed': Domain(m_bed, x=[0.00-2.13], y=[-2.45-0.00]),
 'bed1': Domain(bed1, x=[2.13-3.94], y=[-2.45-0.00]),
 'm_closet': Domain(m_closet, x=[0.00-1.47], y=[-3.33--2.45]),
 'transit': Domain(transit, x=[1.47-2.62], y=[-3.41--2.45]),
 'bed1_closet': Domain(bed1_closet, x=[2.62-3.77], y=[-2.82--2.45]),
 'm_bath': Domain(m_bath, x=[0.00-1.61], y=[-4.57--3.33]),
 'corridor': Domain(corridor, x=[1.61-2.06], y=[-6.12--3.41]),
 'linen': Domain(linen, x=[2.06-2.63], y=[-3.80--3.41]),
 'laundry': Domain(laundry, x=[2.63-3.78], y=[-3.91--2.82]),
 'bath': Domain(bath, x=[0.00-1.58], y=[-5.50--4.57]),
 'den': Domain(den, x=[2.06-3.87], y=[-5.96--3.91]),
 'kitchen': Domain(kitchen, x=[0.00-1.47], y=[-8.17--5.50]),
 'dining': Domain(dining, x=[1.47-3.01], y=[-8.56--5.96]),
 'den_closet': Domain(den_closet, x=[3.01-3.81], y=[-6.32--5.96]),
 'living': Domain(living, x=[0.00-3.16], y=[-10.59--8.17])}

In [6]:
plot_general(doms)

In [6]:
re = Reporter(lout)
re.run()
re.problems

[Problem(ix=1, SIDE_HOLE, ['den', 'laundry'], resolved=False,  n_actions=2,
 Problem(ix=2, SIDE_HOLE, ['bath', 'm_bath'], resolved=False,  n_actions=2,
 Problem(ix=3, SIDE_HOLE, ['laundry', 'bed1_closet'], resolved=False,  n_actions=2,
 Problem(ix=4, SIDE_HOLE, ['kitchen', 'bath'], resolved=False,  n_actions=2,
 Problem(ix=5, SIDE_HOLE, ['m_bath', 'm_closet'], resolved=False,  n_actions=2,
 Problem(ix=6, SIDE_HOLE, ['living', 'den_closet'], resolved=False,  n_actions=2]

In [7]:
c = Cook(re)

Could not report on problems for ActionType.PULL on den
Could not report on problems for ActionType.PULL on kitchen
initializing.. 0
running again -> 1
problem being studied: -> Problem(ix=2, SIDE_HOLE, ['bath', 'm_bath'], resolved=False,  n_actions=2
next best layout m_bath-STRETCH-for-SIDE_HOLE near ['bath', 'm_bath']5-PS:17.29-S:11.15


In [17]:
try:
    c.run_again()
except DomainComparisonError as e:
    err
r

(Domain(linen, x=[2.06-2.63], y=[-3.80--3.41]), Domain(, x=[1.61-2.18], y=[-6.12--3.41]))


DomainComparisonError: Problem domains should have a relationship!

In [16]:
r

NameError: name 'r' is not defined

In [18]:
f = c.plot_all()

len sols = 7. Len indices = 9
